In [7]:
%matplotlib inline
import torch
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import random
import sys
sys.path.append('..')
from d2lzh_pytorch import *

In [10]:
#生成数据集
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.from_numpy(np.random.normal(0,1, (num_examples, num_inputs)))
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.from_numpy(np.random.normal(0, 0.01, size=labels.size()))

# 初始化模型参数
# 权重初始化为均值为0，标准差为0.01的正态随机数
w = torch.tensor(np.random.normal(0, 0.01, (num_inputs, 1)), dtype=torch.double)
#偏差初始化为0
b = torch.zeros(1, dtype=torch.double)
#梯度追踪设为True
w.requires_grad_(requires_grad=True)
b.requires_grad_(requires_grad=True)

lr = 0.03
num_epochs = 3
batch_size = 10
net = linreg
loss = squared_loss
# 训练模型
for epoch in range(num_epochs):
    for x, y in data_iter(batch_size, features, labels):
        # 前向计算
        y_ = net(x, w, b)
        # 计算loss
        l = loss(y_, y).sum() # 调用.sum()使l变成标量
        # 计算loss梯度
        l.backward()
        # 后向传播，优化参数
        sgd([w, b], lr, batch_size)
        #梯度清零
        w.grad.data.zero_()
        b.grad.data.zero_()
        
    train_l = loss(net(features, w, b), labels)
    print('epoch %d, loss %f' % (epoch + 1, train_l.mean().item()))
    
# 对比真实参数和预测参数
print(true_w, '\n', w)
print(true_b, '\n', b)    

epoch 1, loss 0.035409
epoch 2, loss 0.000134
epoch 3, loss 0.000050
[2, -3.4] 
 tensor([[ 1.9996],
        [-3.4000]], dtype=torch.float64, requires_grad=True)
4.2 
 tensor([4.1997], dtype=torch.float64, requires_grad=True)


###### 生成数据集

In [ ]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.from_numpy(np.random.normal(0,1, (num_examples, num_inputs)))
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.from_numpy(np.random.normal(0, 0.01, size=labels.size()))

#### 可视化数据集

In [2]:
def use_svg_display():
    #用矢量图显示
    display.set_matplotlib_formats('svg')
    
def set_figsize(figsize=(3.5, 2.5)):
    use_svg_display()
    #设置图的尺寸
    plt.rcParams['figure.figsize'] = figsize

In [ ]:
set_figsize()
plt.scatter(features[:, 1].numpy(), labels.numpy(), 1);

#### 读取数据

In [3]:
def data_iter(batch_size, features, labels):
    '''
    按batch取数据
    return：每次返回batch_size个随机样本的特征和标签
    '''
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices)
#     print(indices)
    for i in range(0, num_examples, batch_size):
        # j为 batch_size*1 的tensor，
        j = torch.LongTensor(indices[i: min(i+batch_size, num_examples)])
        yield features.index_select(0, j), labels.index_select(0, j)

#### 定义模型

In [4]:
def linreg(x, w, b):
    '''
    线性回归的矢量计算，使用mm函数做矩阵乘法
    '''
    return torch.mm(x, w) + b

#### 定义损失函数

In [5]:
def squared_loss(y_hat, y):
    '''
    使用平方损失来定义线性回归的损失函数
    '''
    return (y_hat - y.view(y_hat.size())) ** 2 / 2

#### 定义优化算法

In [6]:
def sgd(params, lr, batch_size):
    '''
    使用随机梯度下降算法来优化损失函数
    '''
    for param in params:
        # 注意：更改param时使用param.data，这样不会影响梯度传播
        # 自动求梯度模块计算得到的是一个batch的梯度和，将其除以batch_size来得到平均值
        param.data -= lr * param.grad / batch_size

#### 初始化模型参数

In [ ]:
# 权重初始化为均值为0，标准差为0.01的正态随机数
w = torch.tensor(np.random.normal(0, 0.01, (num_inputs, 1)), dtype=torch.double)
#偏差初始化为0
b = torch.zeros(1, dtype=torch.double)
#梯度追踪设为True
w.requires_grad_(requires_grad=True)
b.requires_grad_(requires_grad=True)

#### 训练模型

In [ ]:
lr = 0.03
num_epochs = 3
batch_size = 10
net = linreg
loss = squared_loss

for epoch in range(num_epochs):
    for x, y in data_iter(batch_size, features, labels):
        l = loss(net(x, w, b), y).sum() # 调用.sum()使l变成标量
        l.backward()
        sgd([w, b], lr, batch_size)
        #梯度清零
        w.grad.data.zero_()
        b.grad.data.zero_()
        
    train_l = loss(net(features, w, b), labels)
    print('epoch %d, loss %f' % (epoch + 1, train_l.mean().item()))

In [ ]:
# 对比真实参数和预测参数
print(true_w, '\n', w)
print(true_b, '\n', b)